In [9]:
# Initial imports
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path

%matplotlib inline



In [ ]:
# Reading whale returns
whale_returns_csv = Path("Resources/whale_returns.csv")
whale_returns = pd.read_csv(whale_returns_csv, index_col="Date", parse_dates=True, infer_datetime_format=True)
# whale_returns.describe()
whale_returns.sort_index(ascending = True, inplace = True)
whale_returns.head()



In [ ]:
# Count nulls
print(whale_returns.isnull().sum())
print(whale_returns[whale_returns.isnull().any(axis = 1)])



In [ ]:
# Drop nulls
whale_returns.dropna(inplace=True)
whale_returns.isnull().sum()


In [ ]:
# Reading algorithmic returns
algo_returns_csv = Path("Resources/algo_returns.csv")
algo_returns = pd.read_csv(algo_returns_csv, index_col="Date", parse_dates=True, infer_datetime_format=True)
algo_returns.sort_index(ascending = True, inplace = True)
algo_returns.head(100)


In [ ]:
# Count nulls
algo_returns.isnull().sum()


In [ ]:
# Drop nulls
algo_returns.dropna(inplace = True)
algo_returns.head()


In [ ]:
# Reading S&P 500 Closing Prices, sorting index
sp500_history_csv = Path("Resources/sp500_history.csv")
sp500_history = pd.read_csv(sp500_history_csv, index_col="Date", parse_dates=True, infer_datetime_format=True)
sp500_history.sort_index(ascending = True, inplace = True)
sp500_history.head()


In [ ]:
# Check Data Types
sp500_history.dtypes


In [ ]:
# Fix Data Types
sp500_history["Close"]= sp500_history["Close"].str.replace('$','').astype("float")
sp500_history.sort_index(inplace = True)
sp500_history


In [ ]:
# Calculate Daily Returns
sp500_daily_returns = sp500_history.pct_change()


In [ ]:
# Drop nulls
sp500_daily_returns.dropna(inplace = True)
sp500_daily_returns.head()


In [ ]:
# Rename Column
sp500_daily_returns.rename(columns = {"Close":"sp500"}, inplace = True)


In [ ]:
# Concatenate all DataFrames into a single DataFrame
df_daily = pd.concat([whale_returns, algo_returns,sp500_daily_returns], axis = "columns",join = "inner")


In [ ]:
# Plot daily returns
df_daily.plot(figsize = (20,10))


In [ ]:
# Plot cumulative returns
df_cum = (1+ df_daily).cumprod()
df_cum.plot(figsize = (20,10))


In [ ]:
# Box plot to visually show risk
df_daily.plot(kind = "box", figsize = (20,10))


In [ ]:
# Daily Standard Deviations
# Calculate the standard deviation for each portfolio. Which portfolios are riskier than the S&P 500?
df_daily_std = pd.DataFrame(df_daily.std()).rename(columns = {0:"std"})
df_daily_std


In [ ]:
# Determine which portfolios are riskier than the S&P 500
higher_std = df_daily_std[df_daily_std["std"] > df_daily_std.loc["sp500", "std"]]
higher_std


In [ ]:
# Calculate the annualized standard deviation (252 trading days)
df_annualized_std = df_daily_std*np.sqrt(252)
df_annualized_std


In [ ]:
# Calculate and plot the rolling standard deviation for the S&PP 500 using a 21 day window
sp500_rolling_21 = df_daily[["sp500"]].rolling(window = 21).std()
sp500_rolling_21.plot(figsize = (20,10))


In [ ]:
# Correlation
correlation_sp500 = pd.DataFrame(df_daily.corr()).loc[:,"sp500"]
correlation_sp500[correlation_sp500==correlation_sp500[correlation_sp500<1].max()]


In [ ]:
df_daily.plot(kind="scatter", y = "Algo 2", x = "sp500", figsize = (20,10))
